In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import itertools as ittls
from collections import Counter

In [41]:
class Bi_RNN(object):
    def __init__(self,time_step,input_dim,hidden_size,batch_size,class_num):
        self.time_step = time_step
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.class_num = class_num
        self.rnn_cell = rnn.BasicLSTMCell(hidden_size)
        
        self.input_x = tf.placeholder(dtype=tf.float32,shape=[None,time_step,input_dim])
        self.label_y = tf.placeholder(dtype=tf.float32,shape=[None,class_num])
        
        state = self.rnn_cell.zero_state(batch_size = self.batch_size, dtype = tf.float32)
        
        input_list = tf.unstack(self.input_x,time_step,axis=1)
        self.outputs,_,_ = tf.nn.static_bidirectional_rnn(
                            cell_fw = self.rnn_cell,
                            cell_bw = self.rnn_cell,
                            dtype = tf.float32,
                            inputs = input_list)
        ## the hidden weight must be hidden * 2 due to the bidirectional rnn
        self.softmax_weight = tf.Variable(tf.random_normal([self.hidden_size * 2,self.class_num]))
        self.softmax_bias = tf.Variable(tf.random_normal([self.class_num]))
        print self.outputs[-1].shape
        logits = tf.matmul(self.outputs[-1],self.softmax_weight) + self.softmax_bias
        
        self.prediction = tf.nn.softmax(logits)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                           labels=self.label_y))
        correct_pred = tf.equal(tf.argmax(self.prediction,1),tf.argmax(self.label_y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
        self.init = tf.global_variables_initializer()
        self.inited = False
        self.trained = False
    def batch_gen(self,batch_size,polulation_size=500):
        collection = []
        for _ in ittls.cycle(xrange(polulation_size)):
            collection.append(_)
            if len(collection) >= batch_size:
                yield collection
                collection = []
    def fit(self,sess,x,y,learning_rate=0.001,iterations=500):
        assert x.shape[1:] == self.input_x.shape[1:]
        if not self.inited:
            sess.run(self.init)
            self.inited = True
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(self.cost)
        
        index_gen = self.batch_gen(batch_size = self.batch_size, polulation_size = x.shape[0])
        cost_list = []
        
        for step in xrange(1,iterations+1):
            index = next(index_gen)
            feed_x = x[index]
            feed_y = y[index]
            cost,_ = sess.run([self.cost,train_op],feed_dict={self.input_x:feed_x,self.label_y:feed_y})
            cost_list.append(cost)
            if step % 200 == 0 or step == 1 or step == iterations:
                cost,acc = sess.run([self.cost,self.accuracy],
                                   feed_dict = {self.input_x:feed_x, self.label_y: feed_y})
                print "step: ", step, ", batch_cost= ", "{:.4f}".format(cost),",accuracy=", "{:.3f}".format(acc)
        self.trained = True
    def predict(self,sess,new_x):
        assert new_x.shape[1:] == self.input_x.shape[1:]
        predict = sess.run([tf.argmax(self.prediction,1)],feed_dict = {self.input_x: new_x})
        logits = sess.run(self.prediction,feed_dict = {self.input_x: new_x})
        return predict,logits

In [42]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        x = np.random.random((1000,10,15))
        y = np.random.binomial(n=1, p=0.4, size=(1000,1))
        encoded_y = np.zeros((len(y),2),dtype=np.float32)
        for index,label in enumerate(y):
            encoded_y[index,label] = 1.0
        print y.shape
        bilstm = Bi_RNN(time_step= 10, input_dim = 15, hidden_size = 200, class_num = 2, batch_size = 50)
        bilstm.fit(sess=sess, x=x, y=encoded_y, learning_rate = 0.001,iterations=500)
        prediction = bilstm.predict(sess=sess, new_x = x[1:40])
        print prediction

(1000, 1)
(?, 400)
step:  1 , batch_cost=  0.9104 ,accuracy= 0.440
step:  200 , batch_cost=  0.6524 ,accuracy= 0.660
step:  400 , batch_cost=  0.6512 ,accuracy= 0.620
step:  500 , batch_cost=  0.6509 ,accuracy= 0.640
([array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])], array([[ 0.66579956,  0.33420047],
       [ 0.59355372,  0.40644628],
       [ 0.59546393,  0.4045361 ],
       [ 0.52845246,  0.47154751],
       [ 0.51844227,  0.48155773],
       [ 0.63332444,  0.36667553],
       [ 0.55292606,  0.44707394],
       [ 0.6613555 ,  0.3386445 ],
       [ 0.54862416,  0.4513759 ],
       [ 0.62505394,  0.37494606],
       [ 0.58455068,  0.41544932],
       [ 0.51425821,  0.48574173],
       [ 0.63816571,  0.36183429],
       [ 0.59151661,  0.40848336],
       [ 0.57806844,  0.42193154],
       [ 0.60443234,  0.39556772],
       [ 0.53951347,  0.46048656],
       [ 0.55784518,  0.44215485],
       [ 0.649809